In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.usherbrooke.ca/admission/programme/215/baccalaureat-en-genie-mecanique#structure"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [6]:
driver.find_element_by_id("accordion-000").click()

In [7]:
page_soup = soup(driver.page_source, 'lxml')

In [8]:
tables = page_soup.find("div", {"id": "accordion-000"}).findAll("table", {"class": "table table-striped"})
len(tables)

17

In [9]:
link_texts = [container.text.strip() for table in tables for container in table.findAll("a")]
link_texts

["Mathématiques de l'ingénieur I",
 'Résistance des matériaux I',
 'Résistance des matériaux II',
 "Mathématiques de l'ingénieur II",
 'Résistance des matériaux III',
 'Dynamique des fluides appliquée',
 'Transferts thermiques',
 'Simulation numérique appliquée',
 'Introduction au génie-qualité',
 'Algèbre linéaire',
 'Équations différentielles et calcul intégral en génie',
 'Statique',
 "Exploitation de l'ordinateur I",
 "Exploitation de l'ordinateur II",
 'Dynamique',
 'Introduction à la thermodynamique',
 'Mécanique des fluides',
 'Éléments et systèmes mécaniques',
 'Création de produits innovants',
 'Microstructures et choix des matériaux',
 'Systèmes mécatroniques analogiques',
 'Fiabilité des matériaux',
 "Introduction aux techniques d'usinage",
 'Procédés de mise en forme des matériaux',
 'Systèmes mécatroniques numériques',
 'Projet de mécatronique',
 'Dessin technique',
 'Introduction aux matériaux',
 'Éléments de la méthode expérimentale',
 'Santé et sécurité du travail',
 'A

In [10]:
len(link_texts)

97

In [11]:
len(list(dict.fromkeys(link_texts)))

97

# 2. Test run - try to scrape first course

In [12]:
driver.find_element_by_link_text(link_texts[0]).click()

In [13]:
page_soup = soup(driver.page_source, 'lxml')

In [14]:
course_title = page_soup.find("h1").text.strip()
course_title

"IMC100 - Mathématiques de l'ingénieur I"

In [15]:
course_code = course_title.split(" - ")[0]
course_code

'IMC100'

In [16]:
course_name = course_title.split(" - ")[1]
course_name

"Mathématiques de l'ingénieur I"

In [21]:
course_desc_objective = page_soup.find("h3", text = "Cible(s) de formation").findNextSibling().text.strip().replace("«\xa0", " ").replace("\xa0»", " ").replace("      ", " ")
course_desc_objective

"Être apte à manipuler les concepts mathématiques pertinents aux compétences  Lois de conservation des milieux continus  et  Maîtrise de l'outil expérimental et systèmes mécatroniques ; inculquer des notions de statistique et de probabilités pour planifier les expériences et pour décrire et analyser les résultats expérimentaux."

In [20]:
course_desc_content = page_soup.find("h3", text = "Contenu").findNextSibling().text.strip().replace("\xa0", " ").replace("      ", " ")
course_desc_content

"Variables complexes : opérations élémentaires; forme polaire; exponentielle complexe. Transformée de Laplace: modélisation d'un système mécanique; transformée de Laplace et propriétés; application à la résolution d'E.D. linéaires; représentations temporelle/fréquentielle d'un système. Analyse vectorielle : opérateurs vectoriels; intégrales de ligne, de flux; théorèmes intégraux. Statistique descriptive, modélisation statistique, estimations, prédictions et tests d'hypothèses, plans d'expériences."

In [22]:
driver.back()

# 3. Test run complete. Implement automation script to scrape all courses

In [23]:
course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    link.click()
        
    time.sleep(4)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title = page_soup.find("h1").text.strip()
    course_code = course_title.split(" - ")[0]
    course_name = course_title.split(" - ")[1]
    course_desc_objective = page_soup.find("h3", text = "Cible(s) de formation").findNextSibling().text.strip().replace("«\xa0", " ").replace("\xa0»", " ").replace("      ", " ")
    course_desc_content = page_soup.find("h3", text = "Contenu").findNextSibling().text.strip().replace("\xa0", " ").replace("      ", " ")
    course_desc = course_desc_objective + " " + course_desc_content
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  IMC100 Mathématiques de l'ingénieur I 
 Être apte à manipuler les concepts mathématiques pertinents aux compétences  Lois de conservation des milieux continus  et  Maîtrise de l'outil expérimental et systèmes mécatroniques ; inculquer des notions de statistique et de probabilités pour planifier les expériences et pour décrire et analyser les résultats expérimentaux. Variables complexes : opérations élémentaires; forme polaire; exponentielle complexe. Transformée de Laplace: modélisation d'un système mécanique; transformée de Laplace et propriétés; application à la résolution d'E.D. linéaires; représentations temporelle/fréquentielle d'un système. Analyse vectorielle : opérateurs vectoriels; intégrales de ligne, de flux; théorèmes intégraux. Statistique descriptive, modélisation statistique, estimations, prédictions et tests d'hypothèses, plans d'expériences.
1 courses scraped

Scraped  IMC106 Résistance des matériaux I 
 Présenter les concepts fondamentaux de la mécanique des 

Scraped  ING100 Algèbre linéaire 
 Formuler les problèmes en utilisant les notions de vecteurs et de matrices. Choisir et appliquer les outils appropriés pour résoudre les systèmes linéaires et les problèmes aux valeurs propres. Vecteurs et matrices; exemples; opérations sur les vecteurs; opérations sur les matrices; sous-espaces vectoriels; indépendance linéaire; base, applications. Transformations linéaires : exemples; noyau et image d'une transformation linéaire; applications. Systèmes de n équations linéaires à n inconnues : exemples; méthode d'élimination de Gauss; existence et unicité d'une solution; méthode de la matrice inverse; déterminant et règle de Cramer; techniques numériques; méthode des moindres carrés; applications. Valeurs propres et vecteurs propres : exemples; équations caractéristiques; diagonalisation; techniques numériques d'approximation des valeurs propres; applications.
10 courses scraped

Scraped  ING112 Équations différentielles et calcul intégral en génie 


Scraped  IMC326 Systèmes mécatroniques analogiques 
 Définir un système mécatronique et en identifier les composants, modéliser et analyser un système mécatronique, concevoir l'asservissement d'un système mécatronique et réaliser un asservissement en prototypage rapide. Introduction sur les systèmes mécatroniques, capteurs analogiques, actionneurs, modélisation des systèmes, systèmes dynamiques bouclés, correcteur PID (proportionnel, intégral, dérivé), analyse fréquentielle, et filtrage analogique.
21 courses scraped

Scraped  IMC350 Fiabilité des matériaux 
 Analyser les processus de dégradation des matériaux; approfondir les notions théoriques et appliquées sur les problèmes de défaillance d'origine mécanique ou physicochimique (rupture, corrosion, vieillissement, fatigue, fluage, etc.), afin de faire un choix rationnel et sécuritaire des matériaux, d'apporter des solutions optimales pour la conception et la production, de minimiser la dégradation, de prévoir et prolonger la durée de

Scraped  ING510 Communication en ingénierie 
 Consolider et mûrir la compétence en communication amorcée lors du premier cours de communication qui précède dans le programme. La compétence en rédaction écrite sera consolidée par les apprentissages en argumentation et en vulgarisation scientifique. Elle sera mûrie par l'application à la rédaction de rapports techniques et d'écrits spécifiques. La compétence en échanges oraux sera amorcée et mûrie par l'apprentissage de l'exposé oral. Le niveau de compétence sera atteint par l'apprentissage des attitudes favorisant une bonne communication. Méthodes pour argumenter techniquement. Les règles de l'art pour les rapports et les écrits spécifiques en ingénierie. Les règles de l'art pour l'exposé oral. Les bases de la vulgarisation scientifique. Les attitudes favorisant une bonne communication.
33 courses scraped

Scraped  ING515 Professionnalisme 
 Être apte à choisir et maintenir le style de professionnalisme approprié aux circonstances; être

Scraped  ANS500 Anglais avancé II 
 Acquérir une certaine aisance dans la compréhension et l'expression en anglais : être en mesure de tenir une conversation courante; de comprendre tout genre de textes; de rédiger des textes clairs et structurés. Accent sur la langue parlée : discussions et présentations sur divers sujets d'actualité, mises en situation, idiomatismes; attention particulière à l'écoute active; travail sur du matériel authentique (média, …); lectures variées et exercices de communication écrite dans divers contextes et registres de langues; approfondissement grammatical et lexical.
45 courses scraped

Scraped  ING170 Électricité et magnétisme 
 Appliquer les lois de l'électricité et du magnétisme. Résoudre des circuits résistifs simples. Analyser la réponse de circuits RC et RL simples. Diagramme de propriétés, indice de performance, propriétés électriques, atome, électron, grandeur, unités électriques et magnétiques. Loi d'Ohm, courant, tension, résistance, conductance

Scraped  GMC733 Commande avancée en mécatronique 
 Maîtriser l'ensemble des notions théoriques et pratiques en commande   multivariable non linéaire pour le contrôle de systèmes mécatroniques. Modélisation des systèmes non linéaires multivariables. Linéarisation et   retour linéarisant. Commande par retour d'état. Commande par retour de   sortie. Analyse dans le domaine fréquentiel. Commande robuste.   Identification paramétrique. Commande adaptative.
58 courses scraped

Scraped  GMC734 Dynamique avancée 
 Formuler, résoudre de façon efficace et simuler les équations décrivant le mouvement en 3D de systèmes mécaniques complexes comprenant plusieurs corps rigides et soumis à des contraintes. Géométrie vectorielle algébrique 3D et différentielle. Tenseurs et propriétés de masse. Forces et moments. Équations et contraintes de mouvement. Degrés de libertés. Méthode de Newton-Euler. Conservation de la quantité de mouvement et de l’énergie. Principe de D’Alembert. Relations puissance, travai

Scraped  BGE721 Modélisation en conception d’instruments médicaux 
 Déterminer les comportements et propriétés biologiques pertinents pour le fonctionnement d’un instrument médical. Choisir et mettre en œuvre des techniques de modélisation du vivant et de traitement des signaux pour la conception d’un instrument médical. Valider le fonctionnement d’un instrument médical et les techniques associées dans un environnement contrôlé. Éléments d’anatomie et de physiologie; comportement mécanique ou électrique des tissus biologiques ou organes; modélisation linéaire et non linéaire en bio-ingénierie; détermination de paramètres d’un modèle du vivant; traitement avancé de signaux biomédicaux (représentation par ondelettes, décomposition modale empirique (EMD), filtrage adaptatif); introduction à l’imagerie biomédicale.
72 courses scraped

Scraped  BGM721 Biomécanique du mouvement 
 Résoudre des problématiques en biomécanique du mouvement dans le domaine de la conception de produits, du sport e

Scraped  GBA415 Systèmes mécaniques CVAC 
 Expliquer le fonctionnement des systèmes de chauffage, ventilation et air climatisé (CVAC) et concevoir des éléments de ces systèmes selon les normes en vigueur. Psychométrie. Mécanique des fluides et transfert de chaleur. Introduction à la conception et à l’analyse des systèmes de chauffage, de ventilation et d’air climatisé (CVAC). Données climatiques et charges thermiques. Confort. Diagnostic. Fonctionnement et conception des équipements : systèmes de distribution et de traitement d’air et des fluides, échangeurs et pompes à chaleur, systèmes de réfrigération. Calcul des conduites, grilles, chaudières, convecteurs et systèmes de plomberie. Normes de l'American Society of Heating, Refrigerating and Air-Conditioning Engineers (ASHRAE).
83 courses scraped

Scraped  GMC250 Moteurs à combustion interne à pistons 
 Connaître les principes de la combustion et les utiliser dans la synthèse des cycles Otto et Diesel; acquérir des connaissances génér

Scraped  ACT652 Projet de création d'entreprise technologique 
 Déterminer les actions destinées à mener un projet de création de produit technologique vers un modèle d’affaires d’une entreprise viable, et en encadrer le suivi. Par la voie d’interactions auprès d’un coach : énoncé clair de la raison d’être du projet d’entreprise et des objectifs de performance, planification de l’exécution, suivi des actions de validation permettant de mieux mesurer la viabilité ou non du modèle d’affaires du projet d’entreprise. Apprentissages enseignés et interactions dans le contexte des projets respectifs de création de produit technologique des participantes et participants.
95 courses scraped

Scraped  ACT654 Projet de création d’entreprise technologique II 
 Réviser les éléments d’une équipe performante; évaluer les efforts de validation des composantes du modèle d’affaires, mettre à jour les données de propriété intellectuelle et de veille concurrentielle, repérer les risques du projet; utilise

# 4. Inspect and write to CSV

In [24]:
course_codes

['IMC100',
 'IMC106',
 'IMC113',
 'IMC117',
 'IMC152',
 'IMC211',
 'IMC221',
 'IMC501',
 'IMC510',
 'ING100',
 'ING112',
 'ING225',
 'ING250',
 'ING259',
 'ING260',
 'ING316',
 'ING400',
 'IMC122',
 'IMC155',
 'IMC310',
 'IMC326',
 'IMC350',
 'IMC400',
 'IMC405',
 'IMC451',
 'IMC455',
 'ING211',
 'ING301',
 'ING321',
 'ING525',
 'GIN600',
 'ING500',
 'ING510',
 'ING515',
 'ING600',
 'ING605',
 'ING610',
 'ING615',
 'IMC917',
 'PMC660',
 'PMC760',
 'ANS200',
 'ANS300',
 'ANS400',
 'ANS500',
 'ING170',
 'ING240',
 'ING245',
 'GIN502',
 'GIN503',
 'GIN504',
 'GMC404',
 'GMC443',
 'GMC540',
 'GMC710',
 'GMC712',
 'GMC717',
 'GMC733',
 'GMC734',
 'GMC950',
 'GMC955',
 'GMC140',
 'GMC720',
 'GMC721',
 'GMC729',
 'AMC700',
 'AMC705',
 'GMC747',
 'GMC748',
 'GMC749',
 'GMC758',
 'BGE721',
 'BGM721',
 'BGM722',
 'BGM723',
 'GMC725',
 'GMC732',
 'GMC760',
 'GMC761',
 'GMC762',
 'GMC763',
 'GMC764',
 'GBA415',
 'GMC250',
 'GMC743',
 'GMC750',
 'GMC751',
 'GMC756',
 'GMC757',
 'GMC766',
 'GMC767',

In [25]:
course_names

["Mathématiques de l'ingénieur I",
 'Résistance des matériaux I',
 'Résistance des matériaux II',
 "Mathématiques de l'ingénieur II",
 'Résistance des matériaux III',
 'Dynamique des fluides appliquée',
 'Transferts thermiques',
 'Simulation numérique appliquée',
 'Introduction au génie-qualité',
 'Algèbre linéaire',
 'Équations différentielles et calcul intégral en génie',
 'Statique',
 "Exploitation de l'ordinateur I",
 "Exploitation de l'ordinateur II",
 'Dynamique',
 'Introduction à la thermodynamique',
 'Mécanique des fluides',
 'Éléments et systèmes mécaniques',
 'Création de produits innovants',
 'Microstructures et choix des matériaux',
 'Systèmes mécatroniques analogiques',
 'Fiabilité des matériaux',
 "Introduction aux techniques d'usinage",
 'Procédés de mise en forme des matériaux',
 'Systèmes mécatroniques numériques',
 'Projet de mécatronique',
 'Dessin technique',
 'Introduction aux matériaux',
 'Éléments de la méthode expérimentale',
 'Santé et sécurité du travail',
 'A

In [26]:
course_descs

["Être apte à manipuler les concepts mathématiques pertinents aux compétences  Lois de conservation des milieux continus  et  Maîtrise de l'outil expérimental et systèmes mécatroniques ; inculquer des notions de statistique et de probabilités pour planifier les expériences et pour décrire et analyser les résultats expérimentaux. Variables complexes : opérations élémentaires; forme polaire; exponentielle complexe. Transformée de Laplace: modélisation d'un système mécanique; transformée de Laplace et propriétés; application à la résolution d'E.D. linéaires; représentations temporelle/fréquentielle d'un système. Analyse vectorielle : opérateurs vectoriels; intégrales de ligne, de flux; théorèmes intégraux. Statistique descriptive, modélisation statistique, estimations, prédictions et tests d'hypothèses, plans d'expériences.",
 "Présenter les concepts fondamentaux de la mécanique des milieux continus appliquée aux solides; intégrer ces notions avec les apprentissages provenant des mathémat

In [27]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,IMC100,Mathématiques de l'ingénieur I,Être apte à manipuler les concepts mathématiqu...
1,IMC106,Résistance des matériaux I,Présenter les concepts fondamentaux de la méca...
2,IMC113,Résistance des matériaux II,Calculer les contraintes d’une poutre fabriqué...
3,IMC117,Mathématiques de l'ingénieur II,Reconnaître les différentes classes de problèm...
4,IMC152,Résistance des matériaux III,Définir les principes et notions de base en ré...
...,...,...,...
92,GCH738,Gestion des matières résiduelles,Connaître les principes et maîtriser les outil...
93,GCI736,Analyse du cycle de vie et écoconception,Acquérir une formation de base en écoconceptio...
94,ACT652,Projet de création d'entreprise technologique,Déterminer les actions destinées à mener un pr...
95,ACT654,Projet de création d’entreprise technologique II,Réviser les éléments d’une équipe performante;...


In [28]:
df.to_csv('Sherbrooke_GenieMecanique_Core_and_Electives_Courses.csv', index = False)

In [29]:
driver.quit()